In [ ]:
import os
import gc
import cv2
import math
import time
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from tqdm.auto import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import mean_absolute_error

# =============================================================================
# 1. CONFIGURATION
# =============================================================================
class CFG:
    # --- Paths ---
    BASE_PATH = "/content/dataset"
    train_csv = os.path.join(BASE_PATH, "train.csv")

    # --- Model: DINOv2 Giant (Cực nặng -> Cẩn thận Batch Size) ---
    model_name = "vit_giant_patch14_reg4_dinov2"

    # --- Input Size ---
    # Lưu ý: 756x1512 là rất lớn. Nếu OOM (tràn RAM) thì giảm xuống (518, 1036)
    img_size = (756, 1512)

    # --- Training Params ---
    batch_size = 4
    accumulate_grad = 2   # Tích lũy gradient
    epochs = 20
    FREEZE_EPOCHS = 3

    lr_backbone = 1e-6    # Giant model cần learning rate rất nhỏ
    lr_head = 1e-4
    weight_decay = 1e-4
    seed = 42
    n_fold = 5

    num_workers = 4       # Tăng lên để load ảnh nhanh hơn

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # Thứ tự target phải khớp với logic loss: [Green, Dead, Clover, GDM, Total]
    target_cols = ["Dry_Green_g", "Dry_Dead_g", "Dry_Clover_g", "GDM_g", "Dry_Total_g"]

# =============================================================================
# 2. DATA PREPROCESSING
# =============================================================================
def get_processed_train_df(csv_path: str) -> pd.DataFrame:
    print(f"Loading data from: {csv_path}")
    df_long = pd.read_csv(csv_path)

    df_long["image_path"] = df_long["image_path"].astype(str)
    df_long["image_path_full"] = df_long["image_path"].apply(
        lambda x: os.path.join(CFG.BASE_PATH, x)
    )

    meta_cols = ["image_path", "image_path_full", "Sampling_Date", "State", "Species", "Pre_GSHH_NDVI", "Height_Ave_cm"]
    print("🔁 Pivot Long → Wide...")
    df_wide = df_long.pivot_table(index=meta_cols, columns="target_name", values="target").reset_index()
    df_wide.columns.name = None
    df_wide = df_wide.fillna(0.0)

    for col in CFG.target_cols:
        if col not in df_wide.columns: df_wide[col] = 0.0

    df_wide["group"] = df_wide["Sampling_Date"].astype(str)
    le = LabelEncoder()
    df_wide["stratify_col"] = le.fit_transform(df_wide["State"].astype(str))

    df_wide["fold"] = -1
    sgkf = StratifiedGroupKFold(n_splits=CFG.n_fold, shuffle=True, random_state=CFG.seed)

    for fold, (_, val_idx) in enumerate(sgkf.split(df_wide, df_wide["stratify_col"], groups=df_wide["group"])):
        df_wide.loc[val_idx, "fold"] = fold

    print(f"✅ Data processed! Shape: {df_wide.shape}")
    return df_wide

# =============================================================================
# 3. DATASET
# =============================================================================
class BiomassDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
        self.paths = df['image_path_full'].values
        self.targets = df[CFG.target_cols].values.astype(np.float32)

    def __len__(self): return len(self.df)

    def __getitem__(self, idx):
        path = self.paths[idx]
        img = cv2.imread(path)
        if img is None:
            img = np.zeros((CFG.img_size[0], CFG.img_size[1], 3), dtype=np.uint8)
        else:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        if self.transform:
            img = self.transform(image=img)['image']
        return img, torch.tensor(self.targets[idx])

def get_transforms(data='train'):
    if data == 'train':
        return A.Compose([
            A.Resize(CFG.img_size[0], CFG.img_size[1]),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),

            A.ColorJitter(brightness=0.2, contrast=0.2, p=0.5), # Tăng cường augmentation
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2(),
        ])
    else:
        return A.Compose([
            A.Resize(CFG.img_size[0], CFG.img_size[1]),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2(),
        ])

# =============================================================================
# 4. CUSTOM PHYSICS-INFORMED LOSS
# =============================================================================
class CSIRO_ConsistencyLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.L1Loss() # Dùng MAE Loss cho robust với outlier

    def forward(self, preds, targets):
        # targets columns: [0:Green, 1:Dead, 2:Clover, 3:GDM, 4:Total]

        # 1. Component Loss: Dự đoán đúng từng thành phần
        loss_components = self.l1(preds, targets)

        # 2. Physics Constraint: Green + Dead + Clover + GDM = Total
        # Lấy tổng các thành phần con (4 cột đầu)
        pred_sum_components = torch.sum(preds[:, :4], dim=1)
        pred_total_label = preds[:, 4]

        # Phạt nếu tổng con không bằng tổng cha
        loss_constraint = self.l1(pred_sum_components, pred_total_label)

        # Tổng hợp (Ưu tiên logic toán học)
        return loss_components + 0.5 * loss_constraint

# =============================================================================
# 5. MODEL
# =============================================================================
class BiomassModel(nn.Module):
    def __init__(self, model_name, pretrained=True):
        super().__init__()
        self.backbone = timm.create_model(
            model_name,
            pretrained=pretrained,
            num_classes=0,
            global_pool="token",
            dynamic_img_size=True
        )
        # QUAN TRỌNG: Kích hoạt Checkpointing để không OOM với Giant Model
        self.backbone.set_grad_checkpointing(True)

        self.head = nn.Sequential(
            nn.Linear(self.backbone.num_features, 1024),
            nn.LayerNorm(1024),
            nn.GELU(),
            nn.Dropout(0.3),
            nn.Linear(1024, 5)
        )

    def forward(self, x):
        x = self.backbone(x)
        x = self.head(x)
        return x

# =============================================================================
# 6. TRAINING LOOP
# =============================================================================
def train_loop(fold_idx=0):
    print(f"\n{'='*40}")
    print(f"🚀 START TRAINING FOLD {fold_idx} | DINOv2 GIANT")
    print(f"{'='*40}")

    # 1. Data Setup
    df = get_processed_train_df(CFG.train_csv)
    train_df = df[df['fold'] != fold_idx].reset_index(drop=True)
    valid_df = df[df['fold'] == fold_idx].reset_index(drop=True)
    del df; gc.collect()

    train_loader = DataLoader(BiomassDataset(train_df, get_transforms('train')),
                              batch_size=CFG.batch_size, shuffle=True,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(BiomassDataset(valid_df, get_transforms('valid')),
                              batch_size=CFG.batch_size, shuffle=False,
                              num_workers=CFG.num_workers, pin_memory=True)

    # 2. Init Model
    model = BiomassModel(CFG.model_name, pretrained=True).to(CFG.device)

    # --- Phase 1: Freeze Backbone ---
    print(f"🧊 Phase 1: Freezing Backbone ({CFG.FREEZE_EPOCHS} epochs)")
    for param in model.backbone.parameters(): param.requires_grad = False

    optimizer = AdamW(model.head.parameters(), lr=CFG.lr_head, weight_decay=CFG.weight_decay)
    scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=CFG.epochs, eta_min=1e-7)

    # Thay Loss thường bằng Physics Loss
    criterion = CSIRO_ConsistencyLoss()
    scaler = torch.amp.GradScaler('cuda')

    best_mae = float('inf') # Đánh giá bằng MAE thay vì Loss

    for epoch in range(CFG.epochs):

        # --- Phase 2: Unfreeze Backbone ---
        if epoch == CFG.FREEZE_EPOCHS:
            print(f"🔥 Phase 2: Unfreezing Backbone! Switching to small LR.")
            for param in model.backbone.parameters(): param.requires_grad = True

            # Re-init Optimizer
            optimizer = AdamW([
                {'params': model.backbone.parameters(), 'lr': CFG.lr_backbone},
                {'params': model.head.parameters(), 'lr': CFG.lr_head}
            ], weight_decay=CFG.weight_decay)
            scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=CFG.epochs - epoch, eta_min=1e-7)

        # Training
        model.train()
        train_losses = []
        pbar = tqdm(train_loader, desc=f"Ep {epoch+1}")

        optimizer.zero_grad()
        for step, (imgs, targets) in enumerate(pbar):
            imgs, targets = imgs.to(CFG.device), targets.to(CFG.device)

            with torch.amp.autocast('cuda'):
                outputs = model(imgs)
                loss = criterion(outputs, targets)
                loss = loss / CFG.accumulate_grad # Normalize loss

            scaler.scale(loss).backward()

            if (step + 1) % CFG.accumulate_grad == 0:
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()

            train_losses.append(loss.item() * CFG.accumulate_grad)
            pbar.set_postfix(loss=loss.item() * CFG.accumulate_grad)

        scheduler.step()

        # Validation (Calculate MAE)
        model.eval()
        val_preds, val_targets_all = [], []
        with torch.no_grad():
            for imgs, targets in valid_loader:
                imgs = imgs.to(CFG.device)
                with torch.amp.autocast('cuda'):
                    outputs = model(imgs)

                val_preds.append(outputs.float().cpu().numpy())
                val_targets_all.append(targets.float().numpy())

        val_preds = np.concatenate(val_preds)
        val_targets_all = np.concatenate(val_targets_all)

        # Tính MAE cho từng cột
        mae_scores = []
        for i, col in enumerate(CFG.target_cols):
            mae = mean_absolute_error(val_targets_all[:, i], val_preds[:, i])
            mae_scores.append(mae)

        avg_mae = np.mean(mae_scores)
        avg_train_loss = np.mean(train_losses)

        print(f"Ep {epoch+1} | Train Loss: {avg_train_loss:.4f} | Val MAE (Avg): {avg_mae:.2f}g")
        print(f" > Details: Green:{mae_scores[0]:.1f}, Dead:{mae_scores[1]:.1f}, Total:{mae_scores[4]:.1f}")

        if avg_mae < best_mae:
            best_mae = avg_mae
            torch.save(model.state_dict(), f"best_fold{fold_idx}.pt")
            print(f"🎉 Saved Best Model! MAE: {best_mae:.2f}g")

        gc.collect()

if __name__ == "__main__":
    if os.path.exists(CFG.train_csv):
        train_loop(fold_idx=0)
    else:
        print(f"❌ Check path: {CFG.train_csv}")